# **Practico 5 (parte 4): Entrenar word embeddings**

* [Importación de módulos y librerías](#Importación-de-módulos-y-librerías)
* [Lectura del archivo de mensajes](#Lectura-del-archivo-de-mensajes)
* [Curación del dataset](#Curación-del-dataset)
* [Entrenamiento de los diferentes embeddings (Word2Vec)](#Entrenamiento-de-los-diferentes-embeddings-(Word2Vec))
* [Resumen de información de los word embedding generados](#Resumen-de-información-de-los-word-embedding-generados)
* [Computo de similitud](#Computo-de-similitud)
* [Conclusiones](#Conclusiones)
* [Referencias](#Referencias)

## Importación de módulos y librerías

In [ ]:
# Inclusión de librerias y módulos
import os
import logging
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Usamos las stopwords definidas en la librería nltk más signos de puntuación 
from nltk.corpus import stopwords
stopwords = stopwords.words('english') + [',', "’", '.', ':', '-', ';']

# Algunas utilidades
from utiles import print_some_info
from utiles import convert_emojis
from utiles import convert_emoticons
from utiles import bcolors

# Para convertir str a list
from ast import literal_eval

# Importamos wrod2vec de la librería gensim
from gensim.models import Word2Vec

# Importamos logger para tener informacion de estado
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

data_dir = os.path.join('..', 'dataset')

########################################################
filename = 'yup_messages_preprocessed.csv'
# filename = 'dev_yup_messages_preprocessed.csv'

SAVE_CURATED_DATASET = False

## Lectura del archivo de mensajes
Utilizamos unicamente el archivo de mensajes `yup_messages_preprocessed.csv` debido a que vamos a entrenar un word embedding como `word2vec` y no requerimos de la metadata asociada. Entendemos que para el propósito del análisis y al no utilizar ningún modelo de clasificación o regresión podemos usar el conjunto de datos completo.

In [ ]:
if SAVE_CURATED_DATASET:
    df = pd.read_csv(os.path.join(data_dir, filename))

    print(f'El conjunto de datos utilizado es {filename}')
    print_some_info(df)
else:
    print('Curación evitada')

Curación evitada


## Curación del dataset
Al momento de curar el datset llevamos a cabo los siguientes pasos:

1. Consideramos para el análisis solo las columnas `session_id`, `sent_from` y `text`, en donde la última mencionada contiene los vectores de tokens correspondiente a los turnos de cada diálogo entre estudiante y tutor.
2. Se remueven las filas que contienen mensajes del sistema o que no corresponden a turnos de tutor o estudiante.
3. Se realiza la conversión de tipo del campo texto a lista de strings.
4. Se sustituye el caracter unicode correspondiente a un emoji por un token del tipo `:token_emoji:`
5. Incialmente se consideró la lógica para sustituir la cadena de caracteres correspondiente a un emoticon por un token del tipo `:token_emoticon:`, sin embargo, no lo tratamos porque requiere de una mejor lógica dado que los paréntesis y signos empleados se confunden con los utlizados en ecuaciones y texto regular. Lo que trae aparejado un elevado tiempo de procesamiento.
6. Se convierten los caracteres a minúsculas para unificar los casos con mayúsculas. De este modo Token será convertido a token.
7. Como punto de partida consideramos la lista de stopwords incluidas en el módulo python NLTK con el agregado de los siguientes signos de puntuación [',', "’", '.', ':', '-', ';'] con el objetivo de reducir el  vocabulario a las palabras de mayor utilidad.
8. Se guarda en un archivo .csv el conjunto de datos curado para el presente caso de análisis.
9. Alternativa para cargar el conjunto de datos ya curado y así evitar el tiempo de procesamiento del conjunto de datos crudos.
10. Se repite paso 3 a partir del archivo de datos curados.

In [ ]:
fn = os.path.join(data_dir, filename.replace('.csv','_curated.csv'))
if SAVE_CURATED_DATASET:
    #1. Tomamos solo las columnas que nos pueden servir. Esto es preliminar, podríamos tomar solo `text`
    dfclean = df[['session_id', 'sent_from', 'text']]

    #2. Tomamos solo las filas que sean tutor o student a partir de la columna `sent_from`
    dfclean = dfclean[dfclean.sent_from.isin(['student', 'tutor'])]

    #3. Convertimos a lista de strings el contenido de la columna text
    dfclean['text'] = dfclean.text.apply(lambda x: literal_eval(x))

    #4. Se sustituyen emojis por tokens 
    dfclean['text'] = dfclean.text.apply(lambda x: [convert_emojis(w) for w in x])

    #5. Se sustituyen emoticones por palabras 
    # dfclean['text'] = dfclean.text.apply(lambda x: [convert_emoticons(w) for w in x])

    #6. Convernitimos a minúsculas para unificar el tratamiento
    dfclean['text'] = dfclean.text.apply(lambda x: [w.lower() for w in x])

    #7. Removemos las stopwords
    dfclean['text'] = dfclean.text.apply(lambda x: [w for w in x if w not in stopwords])
    
    #8. Se guarda el dataset curado
    dfclean.to_csv(fn, index=False)
else:
    #9. Se carga el dataset curado 
    dfclean = pd.read_csv(fn)
    
    #10. Convertimos a lista de strings el contenido de la columna text
    dfclean['text'] = dfclean.text.apply(lambda x: literal_eval(x))

print_some_info(dfclean)
print(' ,'.join(dfclean.columns))

El conjunto de datos posee 1411477 filas y 3 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411477 entries, 0 to 1411476
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   session_id  1411477 non-null  int64 
 1   sent_from   1411477 non-null  object
 2   text        1411477 non-null  object
dtypes: int64(1), object(2)
memory usage: 32.3+ MB
None



## Entrenamiento de los diferentes embeddings utilizando Word2Vec

El conjunto de datos de entrenamiento puede ser cargado en memoria, pese a su extensión y, por lo tanto, se alojan en la variable `dfclean`. A los fines de analizar el impacto de algunos de los hiperparámetros hemos generado una estructura de bucles for anidados para iterar una grilla de parámetros y así entrenar diferentes word embeddings. Dentro de los hiperparámetros de interés seleccionamos:

* **size**: (valor predeterminado=100) El número de dimensiones del embedding, por ej. la longitud del vector denso para representar cada token o palabra.
* **window**: (valor predeterminado = 5) La distancia máxima entre una palabra de referencia y las palabras que se encuentran alrededor de esta.
* **min_count**: (valor predeterminado= 5) El recuento mínimo de palabras a considerar al entrenar el modelo, donde las palabras con una ocurrencia menor a dicho valor serán ignoradas.
* **workers**: (valor predeterminado= 3) la cantidad de subprocesos que se utilizarán durante el entrenamiento.
* **sg**: (predeterminado 0 o CBOW) El algoritmo de entrenamiento, ya sea CBOW (0) o salto gramo (1).

La cantidad de epochs se mantuvo en su valor por defecto (5) dado que consideramos que el dataset es extenso.

En base a los diferentes conjuntos de hiperparámetros seleccionados se codifica el nombre del embeding de salida como:

> YYYYMMDD-hhmmss_model_{size}-{window}-{min_count}-{sg}.bin

A modo de ejemplo, en la siguiente celda del presente notebook se puede observar una corrida de entrenamiento. No obstante, en la subsiguiente notebook [Practico_5_part_5.ipynb](https://github.com/giannipablo/MentoriaDiploDatos2020/blob/master/Practico_5_part_5.ipynb) se podrá acceder a un breve análisis del impacto de los diferentes hiperparámetros en algunos de los word embeddings seleccionados.

In [ ]:
size_vect = [100]
sg_vect = [0]
window_vect = [1, 5, 9]
min_count_vect = [300]

model_vect = list()
fnmodel_vect = list()

for sg in sg_vect:
    for size in size_vect:
        for window in window_vect:
            for min_count in min_count_vect:
                # size = 100
                # window = 5
                # min_count = 100
                # sg = 0

                params = f'{size}-{window}-{min_count}-{sg}'
                fnmodel = f'{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}_model_{params}.bin'

                model = Word2Vec(list(dfclean.text), size=size, window=window, min_count=min_count, sg=sg, compute_loss=True, workers=4)
                model.save(fnmodel)
        
                model_vect.append(model)
                fnmodel_vect.append(fnmodel)

2020-10-08 14:23:37,911 : INFO : collecting all words and their counts
2020-10-08 14:23:37,912 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-10-08 14:23:37,942 : INFO : PROGRESS: at sentence #10000, processed 41794 words, keeping 3398 word types
2020-10-08 14:23:37,963 : INFO : PROGRESS: at sentence #20000, processed 84698 words, keeping 5244 word types
2020-10-08 14:23:37,991 : INFO : PROGRESS: at sentence #30000, processed 128313 words, keeping 6816 word types
2020-10-08 14:23:38,014 : INFO : PROGRESS: at sentence #40000, processed 172083 words, keeping 8034 word types
2020-10-08 14:23:38,035 : INFO : PROGRESS: at sentence #50000, processed 214137 words, keeping 9180 word types
2020-10-08 14:23:38,053 : INFO : PROGRESS: at sentence #60000, processed 257601 words, keeping 10338 word types
2020-10-08 14:23:38,073 : INFO : PROGRESS: at sentence #70000, processed 300247 words, keeping 11506 word types
2020-10-08 14:23:38,092 : INFO : PROGRESS: at sentenc

In [ ]:
# fnmodel_vect = ['20201007-003111_model_100-5-100-0.bin',
#                 '20201007-003142_model_1000-5-100-0.bin',
#                 '20201007-003248_model_100-5-100-1.bin',
#                 '20201007-003333_model_1000-5-100-1.bin']
# model_vect = list()

# for fnmodel in fnmodel_vect:
#     model_vect.append(Word2Vec.load(fnmodel))

## Resúmen de información de los word embedding generados

En la siguiente celda de código se pesenta un resúmen de la informacion de los diferentes word embeddings entrenados, se detallan:

* Nombre del archivo correspondiente al word embedding, en el cual se especifican los hiperparámetros utilizados en el mismo
* Tamaño del vocabulario que posee el word embedding
* Las primeras 50 componentes del vocabulario
* Las últimas 50 componentes del vocabulario

In [ ]:
for i, model in enumerate(model_vect):
    print(f'{bcolors.HEADER}{fnmodel_vect[i]} - Tamaño del vocabulario {len(list(model.wv.vocab))}{bcolors.ENDC}')
    print(f'{bcolors.OKGREEN}Primeras 50 componentes del vocabulario{bcolors.ENDC}')
    print(f'{bcolors.OKBLUE}{list(model.wv.vocab.keys())[0:50]}{bcolors.ENDC}')
    print(f'{bcolors.OKGREEN}Ultimas 50 componentes del vocabulario{bcolors.ENDC}')
    print(f'{bcolors.OKBLUE}{list(model.wv.vocab.keys())[-50:]}{bcolors.ENDC}')
    print(f'{bcolors.FAIL}#######################################{bcolors.ENDC}')

20201008-140336_model_100-5-1-0.bin - Tamaño del vocabulario 91791
Primeras 50 compoentes del vocabulario
['<url>', 'hey', 'robert', '!', 'welcome', 'yup', '', 'looking', 'problem', "'ve", 'reviewed', 'finding', 'domain', 'square', 'root', 'function', 'let', "'s", 'work', 'together', 'find', 'exactly', "'re", 'stuck', 'tried', '?', 'simplifying', 'wrong', 'okay', 'actually', 'need', 'simplify', 'however', 'would', "n't", 'please', 'show', 'check', '2x√-x+7', 'appreciate', 'seem', 'relevant', 'discuss', 'move', 'tell', 'mean', 'word', '"', 'meant', 'learned']
Ultimas 50 compoentes del vocabulario
['x/-4', 'x/-4>8', '880÷.20', '80÷20=40', 'johnathon', '3÷6=2', '2but', 'hermela', '738', 'karia', '800+.2', 'antonniets', 'utillities', 'bill(which', '.1×5', 'questoin', 'dook', '37.5×.1=', 'netpay', 'pay(880', '44is', '3,75', '47.75', '×3', '143.25', 'naything', 'z+1)^2', 'z+1)(z+1', '2z+1', '16z+8', '8z+12', '8z+6', '2z+3', '2*x+0', '2(0)+(0)-3(0)+4', '-j-6', '-3j+4', 'upyo', '-4x-16', 'y>7'

## Cómputo de similitud 

A los fines de verificar el adecuado entrenamiento de los diferentes word embeddings y realizar un breve análisis del impacto de los hiperparámetros modificados se computa la similitud respecto a una palabra del vocabulario.

En los resultados se puede observar, cuáles son las 10 palabras que se encuentran más próximas en el vocabulario respecto a la palabra de referencia, en este caso la palabra de referencia seleccionada es `good`.

In [ ]:
for i, model in enumerate(model_vect):
    print(f'{bcolors.HEADER}{fnmodel_vect[i]}{bcolors.ENDC}')
    for ms in model.wv.most_similar("good"):
        print(f'{bcolors.OKBLUE}{ms}{bcolors.ENDC}')
    print(f'{bcolors.FAIL}#######################################{bcolors.ENDC}')

2020-10-08 14:07:22,770 : INFO : precomputing L2-norms of word weight vectors
2020-10-08 14:07:22,885 : INFO : precomputing L2-norms of word weight vectors
2020-10-08 14:07:22,889 : INFO : precomputing L2-norms of word weight vectors


20201008-140336_model_100-5-1-0.bin
('great', 0.8285344839096069)
('nice', 0.6311672925949097)
('awesome', 0.6115093231201172)
('excellent', 0.5722245573997498)
('greatt', 0.5118396282196045)
('perfect', 0.5011483430862427)
('close', 0.4685167074203491)
('well', 0.42632153630256653)
('correct', 0.42467591166496277)
('alright', 0.4242382049560547)
#######################################
20201008-140436_model_100-5-100-0.bin
('great', 0.8119578957557678)
('excellent', 0.6872112154960632)
('nice', 0.6561912894248962)
('awesome', 0.5996153354644775)
('wonderful', 0.5705701112747192)
('amazing', 0.538619875907898)
('fantastic', 0.4992120862007141)
('perfect', 0.4428059160709381)
('nailed', 0.43733787536621094)
('brilliant', 0.4170949459075928)
#######################################
20201008-140513_model_100-5-300-0.bin
('great', 0.8019353151321411)
('nice', 0.677017867565155)
('excellent', 0.6625632047653198)
('amazing', 0.5600944757461548)
('awesome', 0.5501896142959595)
('wonderful', 0.5

## Conclusiones

Si bien el propósito de la presente notebook es el de sistematizar el entrenamiento de diferentes word embeddings, se pueden destacar algunos aspectos de relevancia. Por ejemplo, el hecho de modificar la frecuencia mínima de palabras a considerar, produce embeddings con vocabularios significativamente diferentes. En particular, observamos que las expresiones matemáticas merman sustancialmente en el vocabulario de acuerdo aumenta el umbral de frecuencia mínima requerida. 

Finalmente, al computar la métrica de similitud basada en la distancia coseno para la palabra "good" observamos que el resultado de las diez palabras más similares consistía en palabras con la misma connotacion de sentimiento positivo, a pesar de que la simimlitud decae rapidamente en el rango analizado. Esto nos da un indicio de que el uso de word embeddings para determinar la satisfacción del estudiante podría ser factible.

## Referencias

* https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
* https://radimrehurek.com/gensim/models/word2vec.html